First, let's import needed modules and set random seed (we'll use it if needed).

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split

from linear_regression import SciPyLinearRegressionOLS, LinearRegressionOLS, SciPyLinearRegressionNNOLS
from utils.scaler import StandardScaler

SEED = 42

Loading california housing dataset

In [2]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

Splitting data into training and testing dataset (10% for test)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=True)

Standardize features by removing the mean and scaling to unit variance

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

First solution will use the analytical solution to OLS to get the weights/coefficients: $\hat{\beta} = (X^TX)^{-1}(X^TY)$

In [5]:
ols_reg = LinearRegressionOLS()
ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {ols_reg.coef_}")
print(f"The intercept is {ols_reg.intercept_}")

The coefficients are [ 0.82824645  0.11647235 -0.2672834   0.31051916 -0.0042315  -0.04680151
 -0.90193407 -0.87360011]
The intercept is 2.0670505808570114


Second solution is the sklearn's solution

In [6]:
sklearn_ols_reg = LinearRegression(positive=False)
sklearn_ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {sklearn_ols_reg.coef_}")
print(f"The intercept is {sklearn_ols_reg.intercept_}")

The coefficients are [ 0.82824645  0.11647235 -0.2672834   0.31051916 -0.0042315  -0.04680151
 -0.90193407 -0.87360011]
The intercept is 2.067050580857012


As we can see the results are the same, because sklearn is using <code>scipy.linalg.lstsq</code> method under it's hood. Which, also, wrapped by me below:

In [7]:
scipy_ols_reg = SciPyLinearRegressionOLS()
scipy_ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {scipy_ols_reg.coef_}")
print(f"The intercept is {scipy_ols_reg.intercept_}")

The coefficients are [ 0.82824645  0.11647235 -0.2672834   0.31051916 -0.0042315  -0.04680151
 -0.90193407 -0.87360011]
The intercept is 2.067050580857016


The results are expectedly the same


If we specify <code>positive=False</code> when initializing <code>sklearn.linear_model.LinearRegression</code> object, then sklearn will use <code>scipy.optimize.nnls</code> method

In [8]:
sklearn_ols_reg_pos = LinearRegression(positive=True)
sklearn_ols_reg_pos.fit(X_train_scaled, y_train)
print(f"The coefficients are {sklearn_ols_reg_pos.coef_}")
print(f"The intercept is {sklearn_ols_reg_pos.intercept_}")

The coefficients are [0.82413618 0.23051619 0.         0.0191287  0.0384378  0.
 0.         0.        ]
The intercept is 2.06705058085702


Which is naturally equal to the results from the wrapped method

In [9]:
scipy_nn_ols_reg = SciPyLinearRegressionNNOLS()
scipy_nn_ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {scipy_nn_ols_reg.coef_}")
print(f"The intercept is {scipy_nn_ols_reg.intercept_}")

The coefficients are [0.82413618 0.23051619 0.         0.0191287  0.0384378  0.
 0.         0.        ]
The intercept is 2.06705058085704


When <code>positive</code> set to <code>True</code>, it forces the coefficients to be positive.

We can see that the solution looks completely different - all the coefficients are either zero or positive.

We would want to constrain the coefficients to non-negative values whenever a negative value
makes no physical sense, say because it represents the intensity of a pixel, or the price
of an object, or a frequency count, or a chemical concentration, etc.

TODO:
 - the idea of regularization and why it is needed
 - using gradient descent instead of using formula (analytical solution)
 - comparing results of SGDRegressor with results of ridge regression
 - in which situations using gradient descent is more advisable
 - adjusted R2
 - Linear regression metrics
 - Non-linear methods for regression (decision trees and random forest regressors)

In [10]:
sklearn_sgd_reg = SGDRegressor()
sklearn_sgd_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {sklearn_sgd_reg.coef_}")
print(f"The intercept is {sklearn_sgd_reg.intercept_}")

The coefficients are [-2.55186350e+01  1.15916065e+02  7.20126707e+01 -7.37100789e+01
  1.59321093e+02 -5.68138776e+03  2.84398378e+00  6.64815807e+01]
The intercept is [-36.38129925]


The $R^2$ is the coefficient of determination which is defined as $R^2 = 1 -\frac{\sum\limits _{i = 1} ^N (y_i - \hat y_i)}{\sum\limits _{i = 1} ^N (y_i - \bar y_i)} = 1 - \frac{SS_{fit}}{SS_{mean}}$. <br>
It shows how much the fit of the chosen model is better than a fit of a horizontal straight line (mean value for target $y$). <br>
The best possible score is 1.0, and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of $y$, disregarding the input features, would get a $R^2$ score of 0.0. <br>
$R^2$ can have a negative value without violating any rules of math. $R^2$ is negative only when the chosen model does not follow the trend of the data, so fits worse than a $y$-mean line.

In [11]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
print(f"R2 on the train data is equal to {lr.score(X_train_scaled, y_train).round(2)}")
print(f"R2 on the test data is equal to {lr.score(X_test_scaled, y_test).round(2)}")


R2 on the train data is equal to 0.61
R2 on the test data is equal to 0.6
